In [1]:
import pandas as pd
import numpy as np

### Задание 1 - Напишите функцию, которая классифицирует фильмы

In [2]:
df = pd.read_csv('C:/Users/Lenovo/Downloads/ml-20m/ml-20m/ratings.csv')

In [3]:
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [4]:
def raiting_level(x):
    if x <= 2:
        return 'низкий рейтинг'
    elif x <=4:
        return 'средний рейтинг'
    elif x>4:
        return 'высокий рейтинг'

In [5]:
new_df = df.groupby('movieId').agg({'rating':'mean'}).reset_index().head(5)

In [6]:
new_df['class'] = df.rating.apply(raiting_level)

In [26]:
new_df.head()

,movieId,rating,class
0,1,3.921240,средний рейтинг
1,2,3.211977,средний рейтинг
2,3,3.151040,средний рейтинг
3,4,2.861393,средний рейтинг
4,5,3.064592,средний рейтинг


### Задание 2 - Необходимо написать гео-классификатор

In [8]:
kw = pd.read_csv('C:/Users/Lenovo/Downloads/ml-latest-small/ml-latest-small/keywords.csv')

In [9]:
kw

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195
...,...,...
99995,эльдорадо старый оскол,3705
99996,frigate для firefox,3630
99997,укрсиб,3630
99998,погода в ялте на неделю,3688


In [10]:
geo_data = {'Центр': ['москва', 'тула', 'ярославль'],

            'Северо-Запад': ['петербург', 'псков', 'мурманск'],

            'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

In [11]:
all_cities = sum(geo_data.values(), start = [])

In [12]:
def geo_query (query):
    for x in all_cities:
        if x in query:
            for key, value in geo_data.items():
                if x in value:
                    return key
    return 'undefined'
        

In [13]:
kw['region'] = kw.keyword.apply(geo_query)

In [14]:
kw.query('region != "undefined"')

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
...,...,...,...
99590,авито ярославль автомобили с пробегом,3673,Центр
99634,северпост новости мурманской области,3606,Северо-Запад
99808,полармед мурманск запись на прием,3719,Северо-Запад
99890,яндекс метро москва,4548,Центр


### Задание 3 - Какие фильмы лучше

In [15]:
movie_raiting = pd.read_csv('C:/Users/Lenovo/Downloads/ml-20m/ml-20m/ratings.csv')

In [16]:
movie_name = pd.read_csv('C:/Users/Lenovo/Downloads/ml-20m/ml-20m/movies.csv')

In [18]:
movie_name.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [19]:
merged_df = movie_raiting.merge(movie_name[['movieId', 'title']], on = 'movieId', how = 'left')
merged_df.head()

,userId,movieId,rating,timestamp,title
0,1,2,3.5,1112486027,Jumanji (1995)
1,1,29,3.5,1112484676,"City of Lost Children, The (Cité des enfants p..."
2,1,32,3.5,1112484819,Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
3,1,47,3.5,1112484727,Seven (a.k.a. Se7en) (1995)
4,1,50,3.5,1112484580,"Usual Suspects, The (1995)"


In [27]:
#Создадим список годов в формате string
years = [str(x) for x in np.arange(1950, 2010).tolist()]

In [21]:
def production_year(title_row):
    for year in years:
        if year in title_row:
            return year
    return '1900'

In [28]:
#тестирование функции
production_year('молчание ягнят (2009)')

'2009'

In [23]:
merged_df['year'] = merged_df['title'].apply(production_year)

In [24]:
merged_df.head()

,userId,movieId,rating,timestamp,title,year
0,1,2,3.5,1112486027,Jumanji (1995),1995
1,1,29,3.5,1112484676,"City of Lost Children, The (Cité des enfants p...",1995
2,1,32,3.5,1112484819,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),1995
3,1,47,3.5,1112484727,Seven (a.k.a. Se7en) (1995),1995
4,1,50,3.5,1112484580,"Usual Suspects, The (1995)",1995


In [25]:
merged_df.groupby('year', as_index = False) \
         .agg({'rating':'mean'}) \
         .sort_values(by = 'rating', ascending = False)



,year,rating
8,1957,4.000399
5,1954,3.990345
23,1972,3.953387
13,1962,3.950847
3,1952,3.935292
...,...,...
49,1998,3.429479
51,2000,3.412875
41,1990,3.409707
48,1997,3.376945
